In [1]:
import json
from web3 import Web3
import time


In [2]:
# Connect to an Ethereum node (replace with your own node URL or Infura endpoint)
# web3_provider = 'http://localhost:7545'
web3_provider = "https://base-sepolia-rpc.publicnode.com"

w3 = Web3(Web3.HTTPProvider(web3_provider))

In [3]:
with open("../truffle_compiled_contract/key.json") as f:
    keys = json.load(f)

In [4]:
if w3.is_connected():
    print("Successfully connected to the node at", web3_provider)
else:
    print("Connection to node failed. Please check the URL and try again.")


Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


In [5]:
# Your Ethereum account private key (NEVER share or commit this!)
# private_key = '0xb99052926180cf5daff5b20633add14f33655b4711ede3c3d4ddb40751dc4311'

private_key = keys["privateKey"]



In [6]:
#collection json
with open(r'..\truffle_compiled_contract\build\contracts\CollectionContract.json') as f:
    collection_json = json.load(f)

with open(r'..\truffle_compiled_contract\build\contracts\NFTContract.json') as f:
    nft_json = json.load(f)


with open(r'..\truffle_compiled_contract\build\contracts\NFTAccessControl.json') as f:
    NFTAccessControl_json = json.load(f)


with open(r'..\truffle_compiled_contract\build\contracts\NFTMetadata.json') as f:
    NFTMetadata_json = json.load(f)
    

with open(r'..\truffle_compiled_contract\build\contracts\MasterAccessControl.json') as f:
    MasterAccessControl_json = json.load(f)

In [7]:
# with open(r'..\truffle_compiled_contract\build\contractAddresses\development_addresses.json') as f:
#     address_json= json.load(f)

with open(r'..\truffle_compiled_contract\build\contractAddresses\base_addresses.json') as f:
    address_json= json.load(f)

In [8]:
address_json

{'MasterAccessControl': '0xEAD39C0363378B3100cB8C89820f71353136EBd0',
 'NFTAccessControl': '0x2c6993608197B40ae0d0D1042829541067ac761e',
 'NFTMetadata': '0x13846e6fDe06853f6CC822A58f97AdbEbF1e6AFd',
 'NFTContract': '0xAc537d070AcfA1F0C6df29a87b5d63c26Fff6DcE',
 'CollectionContract': '0x536446035eF24cb011a3B55f0627df2Fad083F67'}

In [9]:
w3 = Web3(Web3.HTTPProvider(web3_provider))

def write_to_contract(contract, function_name: str, 
                      arguments: list, private_key: str, 
                      web3_provider: str,
                      gas = 2000000):
    """
    Write data to a contract function
    :param contract: The contract object
    :param function_name: The name of the function to call
    :param arguments: The arguments to pass to the function
    :param private_key: The private key of the account calling the function
    :param web3_provider: The URL of the Ethereum node
    :return: The transaction receipt
    """
    
    global w3
    
    if w3.is_connected():
        print("Successfully connected to the node at", web3_provider)
    else:
        w3 = Web3(Web3.HTTPProvider(web3_provider))

        # Checking if the connection to the node was successful
        if w3.is_connected():
            print("Successfully connected to the node at", web3_provider)
        else:
            print("Connection to node failed. Please check the URL and try again.")

    account = w3.eth.account.from_key(private_key)

    # Write data to the function
    nonce = w3.eth.get_transaction_count(account.address)
    transaction = contract.functions[function_name](*arguments).build_transaction({
        "from": account.address,
        "nonce": nonce,
        "gas": gas,
        "gasPrice": w3.eth.gas_price
    })
    signed_txn = account.sign_transaction(transaction)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    return tx_receipt



In [10]:
# Your Ethereum address
account = w3.eth.account.from_key(private_key)

collection_contract = w3.eth.contract(address=address_json["CollectionContract"], abi=collection_json["abi"])

# Your Ethereum address
account = w3.eth.account.from_key(private_key)

# collection_contract = w3.eth.contract(address=address_json["CollectionContract"], abi=collection_json["abi"])
nft_contract = w3.eth.contract(address=address_json["NFTContract"], abi=nft_json["abi"])
nft_metadata_contract = w3.eth.contract(address=address_json["NFTMetadata"], abi=NFTMetadata_json["abi"])
nft_access_control_contract = w3.eth.contract(address=address_json["NFTAccessControl"], abi=NFTAccessControl_json["abi"])
master_access_control_contract = w3.eth.contract(address=address_json["MasterAccessControl"], abi=MasterAccessControl_json["abi"])

# Collection

In [11]:
def create_collection(collection):
    

        
    # write to contract
    
    
    function_name = "createCollection"
    arguments = [collection['name'], collection['contextWindow'], collection['model'], collection['image'], collection['description']]
    key = private_key
    
    
    try :
        tx_receipt = write_to_contract(collection_contract, function_name, arguments, key, web3_provider)
        create_event = collection_contract.events.CollectionCreated().process_receipt(tx_receipt)[0]
        collection_id = create_event['args']['collectionId']
        return collection_id
    except:
        print("Error creating collection : ", collection['name'])

In [12]:


def getAllCollections():
    try:
        
        total_collections = collection_contract.functions.getAllCollections().call()
        
        all_collections = []
        counter = 1
        for collection_data in total_collections:
            # collection_data = collection_contract.functions.getCollectionMetadata(i)
            i = counter
            counter += 1
            formatted_collection = {
                "id": i,
                "name": collection_data[0],
                "contextWindow": collection_data[1],
                "model": collection_data[2],
                "image": collection_data[3],
                "description": collection_data[4],
                "creator": collection_data[5],
                "date": collection_data[6],
                "owner": collection_data[7],
                "collectionaddress": f"#{i}",
                # "noOfNFTs": collection_contract.functions.getCollectionNFTCount(i)
            }
            all_collections.append(formatted_collection)

        return all_collections
    
    except Exception as e:
        print(f"Error getting all collections: {str(e)}")
        return None





def getAllCollections_by_address(address):
    try:
        all_collections = getAllCollections()
        return [collection for collection in all_collections if collection['owner'] == address]
    except Exception as e:
        print(f"Error getting collections for address {address}: {str(e)}")
        return None



def get_collection_details_by_id(collection_id):
    try:
        metadata = collection_contract.functions.getCollectionMetadata(collection_id).call()
        nft_count = collection_contract.functions.getCollectionNFTCount(collection_id).call()
        owner = collection_contract.functions.getCollectionOwner(collection_id).call()
        unique_holders = collection_contract.functions.getCollectionUniqueHolders(collection_id).call()


        collection_details = {
            "id": collection_id,
            "name": metadata[0],
            "contextWindow": metadata[1],
            "baseModel": metadata[2],
            "image": metadata[3],
            "description": metadata[4],
            "creator": metadata[5],
            "dateCreated": metadata[6],
            "owner": owner,
            "collectionaddress": f"#{collection_id}",
            "noOfNFTs": nft_count,
            "uniqueHolders": unique_holders,
            "model": metadata[2],
            "noOfServers": 5
        }

        return collection_details
    except Exception as e:
        print(f"Error getting details for collection ID {collection_id}: {str(e)}")
        return None





In [13]:
with open('./data/all_collections.json') as f:
    collections = json.load(f)

In [ ]:
# import sleep
from time import sleep

In [16]:
# !!!!!!!! Uncomment this to create collections !!!!!!!!

for collection in collections["collections"]:
    
    confirmation = input (f"Create collection {collection['name']} ? (y/n) : ")
    
    if confirmation.lower() != "y":
        continue
    
    collection_id = create_collection(collection)
    print(f"Collection ID: {collection_id}")

Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 1
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 2
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 3
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 4
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 5
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 6
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 7
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 8
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 9
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Collection ID: 10
Successfully connected to the node at https://bas

In [17]:
getAllCollections()

[{'id': 1,
  'name': 'Neural Mint Hub',
  'contextWindow': 4096,
  'model': 'Llama 3.1 70B',
  'image': 'https://base.backend.neuranft.com/image/temp7.png',
  'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'date': 1729638466,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'collectionaddress': '#1'},
 {'id': 2,
  'name': 'Quantum Rewards',
  'contextWindow': 2048,
  'model': 'Llama 3.1',
  'image': 'https://base.backend.neuranft.com/image/temp2.png',
  'description': 'Unlock quantum leaps in rewards with our AI-enhanced NFT collection. Each piece represents the convergence of quantum computing concepts and digital art.',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'date': 1729638476,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'collectionaddress': '#2'},

In [18]:
get_collection_details_by_id(1)

{'id': 1,
 'name': 'Neural Mint Hub',
 'contextWindow': 4096,
 'baseModel': 'Llama 3.1 70B',
 'image': 'https://base.backend.neuranft.com/image/temp7.png',
 'description': 'Discover the fusion of neural networks and blockchain at the Neural Mint Hub. This collection offers unique AI-generated NFTs crafted by advanced neural models.',
 'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
 'dateCreated': 1729638466,
 'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
 'collectionaddress': '#1',
 'noOfNFTs': 0,
 'uniqueHolders': 0,
 'model': 'Llama 3.1 70B',
 'noOfServers': 5}

In [ ]:
getAllCollections_by_address("0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612")

# Check Access

In [19]:
master_access_control_contract.functions.hasAccess(address_json["NFTAccessControl"], address_json["NFTContract"]).call()

True

In [20]:
address_json["NFTContract"]

'0xAc537d070AcfA1F0C6df29a87b5d63c26Fff6DcE'

# NFT Cretion

In [25]:
def burn_NFT(collection_id, nft_id):
            
    # write to contract
    function_name       = "burnNFT"
    
    arguments = [collection_id, nft_id]
    key = private_key
    
    try :
        tx_receipt      = write_to_contract(nft_contract, function_name, arguments, key, web3_provider, gas = 4000000)
        print("Completed writing to contract")
        burn_event      = nft_contract.events.NFTBurned().process_receipt(tx_receipt)[0]
        collection_id   = burn_event['args']['collectionId']
        nft_id          = burn_event['args']['tokenId']
        print (f"nft {nft_id} burned in collection {collection_id}")
        return nft_id
    
    except Exception as e:
        print (f"Error burning nft : {nft_id} : {str(e)}")
        





def create_NFT(nft_data):
    

        
    # write to contract
    
    
    function_name       = "createNFT"
    
    _collectionId       =  nft_data['collection_id']
    _name               = nft_data['name']
    level_of_ownership  = 6
    
    
    arguments = [_collectionId, _name, level_of_ownership]
    key = private_key
    
    

    try :
        print(f"Arguments for {_name}: ", arguments)
        tx_receipt = write_to_contract(nft_contract, function_name, arguments, key, web3_provider, gas = 4000000)
        print("Completed writing to contract")
        create_event = nft_contract.events.NFTCreated().process_receipt(tx_receipt)[0]
        collection_id = create_event['args']['collectionId']
        nft_id = create_event['args']['tokenId']
        cretor = create_event['args']['creator']
        
        print (f"nft {nft_id} created by {cretor} in collection {collection_id}")
        
        return nft_id
    except:
        print("Error creating nft : ", nft_data['name'])
        

def create_metadata(nft_data):
    

        
    # write to contract
    
    
    function_name = "createMetadata"
    
    _collectionId = nft_data['collection_id']
    _nftId = nft_data['id']
    # _metadata = {
    #     "image" : nft_data['image'],
    #     "model" : nft_data['model'],
    #     "data"  : "http//localhost:5500/data",
    #     "rag"   : "http//localhost:5500/rag",
    #     "fineTuneData" : "localhost:5500/fineTuneData",
    #     "description" : nft_data['description']
        
    # }
    
    
    _metadata = [
        nft_data['image'],
        nft_data['model'],
        "https://base.backend.neuranft.com/data",
        "https://base.backend.neuranft.com/rag",
        "https://base.backend.neuranft.com/fineTuneData",
        nft_data['description']
    ]
    
    
    
    arguments = [_collectionId, _nftId, _metadata]
    key = private_key
    
    
    try :
        tx_receipt = write_to_contract(nft_metadata_contract, function_name, arguments, key, web3_provider, gas = 4000000)
        create_event = nft_metadata_contract.events.MetadataCreated().process_receipt(tx_receipt)[0]
        collection_id = create_event['args']['collectionId']
        nft_id = create_event['args']['nftId']
        metadata = create_event['args']['metadata']
        
        # print (f"nft {nft_id} created by {cretor} in collection {collection_id}")
        
        print (f"metadata for nft {nft_id} created in collection {collection_id}")
        
        return nft_id
    except:
        print("Error creating nft metadata: ", nft_data['name'])
        
def provide_access(nft_data):
    
    
    if "accessList" not in nft_data:
        return 0
    
    addresses =[
        "0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912",
        "0x3006b0495B8dC916b9b48d1fa90f843fD68f35a2",
        "0x08aebe3C3A8B568E8071C567f7D81fcD64af47d5",
        "0x05D5724EDB06eCeC911C2069043C0Cb6b49c09b1",
        "0x10a655f5754642A58F6e2cc24136081020d21c26"
    ]
    
    _access_list_temp = []
    count_temp = len(addresses)
    
    for i in nft_data['accessList']:
        if count_temp == 0:
            break
        _access_list_temp.append({ "address":addresses[len(addresses)-count_temp],  "accessLevel":i["accessLevel"]})
        count_temp = count_temp - 1
        
    _access_list = _access_list_temp
    
    
    function_name = "grantAccess"
    
    _collectionId = nft_data['collection_id']
    _nftId = nft_data['id']
    
    key = private_key
    
    
    print("Providing access for nft : ", nft_data['name'])
    
    for access in _access_list:
        
        _user = access['address']
        _level = access['accessLevel']
        
        arguments = [_collectionId, _nftId, _user, _level]
        
        try :
            
            
            tx_receipt = write_to_contract(nft_access_control_contract, function_name, arguments, key, web3_provider)
            create_event = nft_access_control_contract.events.AccessGranted().process_receipt(tx_receipt)[0]
            collection_id = create_event['args']['collectionId']
            nft_id = create_event['args']['nftId']
            user = create_event['args']['user']
            accesslevel = create_event['args']['accessLevel']
            
            
            print("\t\t Access granted to user : ", user , " with access level : ", accesslevel)
            
            # print (f"nft {nft_id} created by {cretor} in collection {collection_id}") 
            # return nft_id
        except:
            print("\t\t Error granding user : ", _user) 
        
        
        
        
    
        


In [22]:
with open('./data/all_nft.json') as f:
    all_nft = json.load(f)

In [23]:
all_nft

{'nfts': [{'id': 1,
   'name': 'Cognitive Llama',
   'collection': 'Neural Mint Hub',
   'collection_id': 1,
   'image': 'https://base.backend.neuranft.com/image/temp11.jpg',
   'model': 'Llama 3.1',
   'attributes': [{'trait_type': 'Model', 'value': 'Llama 3.1'},
    {'trait_type': 'Context Window', 'value': '4096 tokens'},
    {'trait_type': 'Total Access', 'value': '1'},
    {'trait_type': 'Collection', 'value': 'Neural Mint Hub'}],
   'accessList': [{'address': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
     'accessLevel': 6},
    {'address': 'TThwHjyzx2E1TNN4GdSoaHeCRJVi3EuKS7', 'accessLevel': 2},
    {'address': 'TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK', 'accessLevel': 2},
    {'address': 'TVh65XCYkerBch3WtgY9kwLqrtH64MhiFe', 'accessLevel': 3},
    {'address': 'TMVQGm1qAQYVdetCeGRRkTWYYrLXuHK2HC', 'accessLevel': 1}],
   'tokenId': '1234',
   'tokenStandard': 'ERC-721',
   'owner': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
   'chain': 'Ethereum',
   'description': 'Cognitive Llama is an advance

In [24]:
for nft in all_nft["nfts"]:
    
    confirmation = input (f"Create NFT {nft['name']} ? (y/n) : ")
    
    if confirmation.lower() != "y":
        continue
    
    nft_id = create_NFT(nft)
    nft['id'] = nft_id
    print(f"NFT ID: {nft_id}")

Arguments for Cognitive Llama:  [1, 'Cognitive Llama', 6]
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x62d67cc95a1313583d5ec244fa873df44eff812cbdc1504b80eae419d9bcae82') and logIndex: 45 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x62d67cc95a1313583d5ec244fa873df44eff812cbdc1504b80eae419d9bcae82') and logIndex: 46 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x62d67cc95a1313583d5ec244fa873df44eff812cbdc1504b80eae419d9bcae82') and logInde

Completed writing to contract
nft 1 created by 0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612 in collection 1
NFT ID: 1
Arguments for Cyborg Bored Ape:  [1, 'Cyborg Bored Ape', 6]
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xdef8f4d80437e3a5d37d83df801041de1dbf70680b1211c6a4753ccfe640c52f') and logIndex: 17 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xdef8f4d80437e3a5d37d83df801041de1dbf70680b1211c6a4753ccfe640c52f') and logIndex: 18 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xdef8f4d80437e3a5d37d83df801041de1dbf70680b1211c6a4753ccfe640c52f') and logInde

Completed writing to contract
nft 2 created by 0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612 in collection 1
NFT ID: 2
Arguments for Thinker Primate:  [1, 'Thinker Primate', 6]
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


...

f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x17756c4f7101bda67e6e359920c6f385a9abf894583cae2dca5fec33a0fe3385') and logIndex: 28 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x17756c4f7101bda67e6e359920c6f385a9abf894583cae2dca5fec33a0fe3385') and logIndex: 29 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x17756c4f7101bda67e6e359920c6f385a9abf894583cae2dca5fec33a0fe3385') and logInde

Completed writing to contract
nft 5 created by 0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612 in collection 1
NFT ID: 5
Arguments for Corrupted Cyborg:  [1, 'Corrupted Cyborg', 6]
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xc97b8b9035c062cfffa05bc023382652290bb1cc21726ce59d1218a33df0d54a') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xc97b8b9035c062cfffa05bc023382652290bb1cc21726ce59d1218a33df0d54a') and logIndex: 2 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xc97b8b9035c062cfffa05bc023382652290bb1cc21726ce59d1218a33df0d54a') and logIndex:

Completed writing to contract
nft 6 created by 0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612 in collection 1
NFT ID: 6
Arguments for Skynet:  [1, 'Skynet', 6]
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Completed writing to contract
nft 7 created by 0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612 in collection 1
NFT ID: 7


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xe54b3fe52174ad4c08f566f07469eb5945ebc563172b0618219c01f18febd197') and logIndex: 39 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xe54b3fe52174ad4c08f566f07469eb5945ebc563172b0618219c01f18febd197') and logIndex: 40 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xe54b3fe52174ad4c08f566f07469eb5945ebc563172b0618219c01f18febd197') and logInde

In [26]:
with open('./data/all_nft_with_id.json',"w") as f:
    json.dump(all_nft, f)

In [27]:
with open('./data/all_nft_with_id.json') as f:
    all_nft = json.load(f)

In [28]:
all_nft

{'nfts': [{'id': 1,
   'name': 'Cognitive Llama',
   'collection': 'Neural Mint Hub',
   'collection_id': 1,
   'image': 'https://base.backend.neuranft.com/image/temp11.jpg',
   'model': 'Llama 3.1',
   'attributes': [{'trait_type': 'Model', 'value': 'Llama 3.1'},
    {'trait_type': 'Context Window', 'value': '4096 tokens'},
    {'trait_type': 'Total Access', 'value': '1'},
    {'trait_type': 'Collection', 'value': 'Neural Mint Hub'}],
   'accessList': [{'address': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
     'accessLevel': 6},
    {'address': 'TThwHjyzx2E1TNN4GdSoaHeCRJVi3EuKS7', 'accessLevel': 2},
    {'address': 'TJfqnAUwX2ViGTP1J81itXWhuJhUYa8RwK', 'accessLevel': 2},
    {'address': 'TVh65XCYkerBch3WtgY9kwLqrtH64MhiFe', 'accessLevel': 3},
    {'address': 'TMVQGm1qAQYVdetCeGRRkTWYYrLXuHK2HC', 'accessLevel': 1}],
   'tokenId': '1234',
   'tokenStandard': 'ERC-721',
   'owner': 'TLTtQz2Njap4nPAQGPGJn5EEksoM1uw4Qr',
   'chain': 'Ethereum',
   'description': 'Cognitive Llama is an advance

In [29]:
for nft in all_nft["nfts"]:
    
    confirmation = input (f"Create NFT Metadata {nft['name']} with id {nft['id']} ? (y/n) : ")
    if confirmation.lower() != "y":
        continue
    nft_id = create_metadata(nft)
    print (f"Metadata created for nft {nft_id}")

Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 1 created in collection 1
Metadata created for nft 1
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 2 created in collection 1
Metadata created for nft 2
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 3 created in collection 1
Metadata created for nft 3
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 1 created in collection 2
Metadata created for nft 1
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 2 created in collection 2
Metadata created for nft 2
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 4 created in collection 1
Metadata created for nft 4
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
metadata for nft 5 created in coll

In [30]:
for nft in all_nft["nfts"]:
    
    if "accessList" not in nft or len(nft["accessList"]) == 0:
        continue
         
    confirmation = input (f"Provide access for NFT {nft['name']} with id {nft['id']} ? (y/n) : ")
    
    nft_id = provide_access(nft)
    
    print (f"Access provided for nft {nft_id}")

Providing access for nft :  Cognitive Llama
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xe1871ccca87d4ac378ff32fc05b2c107bcb8de869c88bdf4e62880884ed39f63') and logIndex: 43 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  6
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x77aed38dcdea586e37f0aa6bfa8de3d0ede249054a8ada9074b47c719d0c7385') and logIndex: 22 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


		 Access granted to user :  0x3006b0495B8dC916b9b48d1fa90f843fD68f35a2  with access level :  2
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


...

f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xbe3d9b234bb480c0a6f2fd2a646febad34328ed70c1e7acc01b0ec1f3cbc09ec') and logIndex: 22 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  1
Access provided for nft None
Providing access for nft :  Corrupted Cyborg
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xf1a84018a23bf0cbc4a45240c0fedbcb8d01169a76544eafa8f78ca022cdf75e') and logIndex: 30 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  1
Access provided for nft None
Providing access for nft :  Skynet
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
		 Access granted to user :  0xcb6d7cDCa0563575b6b734FA4f3e9d6ac7542912  with access level :  1
Access provided for nft None


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xcf1f4291e0c66863e9e55c1f260cb9ddab2106d580f09abfcb331f447ca1023b') and logIndex: 42 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


In [70]:
collection_id_to_burn = int(input("Burn NFT: Please enter the collection ID:"))
nft_id_to_burn = int(input("Burn NFT: Please enter the NFT ID:"))
confirmations = input(f"Burn NFT{nft_id_to_burn}, Collection{collection_id_to_burn} : Please type y or Yes for confirmations:")
if confirmations.lower() == "y" or confirmations.lower() == "yes":
    print("Burning NFT...")
    burn_NFT(collection_id_to_burn, nft_id_to_burn)
else:
    print("Burn NFT: Operation cancelled")

Burning NFT...
Successfully connected to the node at https://base-sepolia-rpc.publicnode.com
Completed writing to contract
nft 2 burned in collection 2


f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xcbb4557d9d7c8f347d45b44c1a3f2ab0be78f80b618ac0da48af9015eeda5022') and logIndex: 8 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xcbb4557d9d7c8f347d45b44c1a3f2ab0be78f80b618ac0da48af9015eeda5022') and logIndex: 9 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
f:\Environments\.conda\envs\truffle\Lib\site-packages\web3\contract\base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xcbb4557d9d7c8f347d45b44c1a3f2ab0be78f80b618ac0da48af9015eeda5022') and logIndex:

In [31]:
all_nft_of_a_collection_basic(2)

NameError: name 'all_nft_of_a_collection_basic' is not defined

## Contract View Functions

In [32]:
def all_nft_information_basic(nft_id, collection_id):
    try:
        nft_info = nft_contract.functions.getNFTInfo(collection_id, nft_id).call()
        
        return {
            "id": nft_id,
            "collectionId": collection_id,
            "levelOfOwnership": nft_info[0],
            "name": nft_info[1],
            "creator": nft_info[2],
            "creationDate": nft_info[3],
            "owner": nft_info[4],
        }
    except Exception as e:
        print(f"Error getting information for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None

def all_nft_information(nft_id, collection_id):
    try:
        nft_info = nft_contract.functions.getNFTInfo(collection_id, nft_id).call()
        metadata = nft_metadata_contract.functions.getMetadata(collection_id, nft_id).call()
        
        return {
            "id": nft_id,
            "collectionId": collection_id,
            "levelOfOwnership": nft_info[0],
            "name": nft_info[1],
            "creator": nft_info[2],
            "creationDate": nft_info[3],
            "owner": nft_info[4],
            "metadata": {
                "image": metadata[0],
                "baseModel": metadata[1],
                "data": metadata[2],
                "rag": metadata[3],
                "fineTuneData": metadata[4],
                "description": metadata[5]
            }
        }
    except Exception as e:
        print(f"Error getting information for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None


def all_nft_of_a_collection_basic(collection_id):
    try:
        nft_count = nft_contract.functions.getCollectionNFTCount(collection_id).call()
        nft_ids = nft_contract.functions.getCollectionNFTs(collection_id).call()
        
        nfts = []
        for nft_id in nft_ids:
            nft_info = all_nft_information_basic(nft_id, collection_id)
            if nft_info:
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for collection {collection_id}: {e}")
        return None


def all_nft_of_a_collection(collection_id):
    try:
        nft_count = nft_contract.functions.getCollectionNFTCount(collection_id).call()
        nft_ids = nft_contract.functions.getCollectionNFTs(collection_id).call()
        
        nfts = []
        for nft_id in nft_ids:
            nft_info = all_nft_information(nft_id, collection_id)
            if nft_info:
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for collection {collection_id}: {e}")
        return None

def all_access_levels_of_a_collection_nft(collection_id, nft_id):
    try:
        users_access = nft_access_control_contract.functions.getAllUsersAccessForNFT(collection_id, nft_id).call()
        
        access_levels = []
        for user_access in users_access:
            access_levels.append({
                "user": user_access[0],
                "accessLevel": user_access[1]
            })
        
        return access_levels
    except Exception as e:
        print(f"Error getting access levels for NFT ID {nft_id} in collection {collection_id}: {e}")
        return None

def all_nfts_own_or_have_access_by_user(user_address):
    try:
        user_access_entries = nft_access_control_contract.functions.getAllAccessForUser(user_address).call()
        
        nfts = []
        for entry in user_access_entries:
            collection_id, nft_id, access_level = entry
            nft_info = all_nft_information(nft_id, collection_id)
            if nft_info:
                nft_info['accessLevel'] = access_level
                nfts.append(nft_info)
        
        return nfts
    except Exception as e:
        print(f"Error getting NFTs for user {user_address}: {e}")
        return None


In [33]:
all_nft_information(1, 1)

{'id': 1,
 'collectionId': 1,
 'levelOfOwnership': 6,
 'name': 'Cognitive Llama',
 'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
 'creationDate': 1729638660,
 'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
 'metadata': {'image': 'https://base.backend.neuranft.com/image/temp11.jpg',
  'baseModel': 'Llama 3.1',
  'data': 'https://base.backend.neuranft.com/data',
  'rag': 'https://base.backend.neuranft.com/rag',
  'fineTuneData': 'https://base.backend.neuranft.com/fineTuneData',
  'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'}}

In [34]:
all_nft_of_a_collection(1)

[{'id': 1,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cognitive Llama',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'creationDate': 1729638660,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'metadata': {'image': 'https://base.backend.neuranft.com/image/temp11.jpg',
   'baseModel': 'Llama 3.1',
   'data': 'https://base.backend.neuranft.com/data',
   'rag': 'https://base.backend.neuranft.com/rag',
   'fineTuneData': 'https://base.backend.neuranft.com/fineTuneData',
   'description': 'Cognitive Llama is an advanced AI-driven bot, utilizing the power of Llama 3.1 with multi-level reasoning and cognition. Part of the Neural Mint Hub collection.'}},
 {'id': 2,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cyborg Bored Ape',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'creationDate': 1729638666,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'metadata': {'image': 'https://base.backend.neuranft.com/image/temp10

In [ ]:
all_access_levels_of_a_collection_nft(1,1)

In [ ]:
all_nfts_own_or_have_access_by_user("0x53E5dA1dC6D0C6ef7e85ed52C2A256dd1AA669bd")


In [28]:
all_nft_information_basic(1)

6

In [29]:
nft_contract.functions.getCollectionNFTCount(2).call()

2

In [38]:
all_nft_of_a_collection_basic(1)

[{'id': 2,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Cognitive Llama',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'creationDate': 1729267704,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612'},
 {'id': 3,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Dark Llama',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'creationDate': 1729267716,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612'},
 {'id': 4,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Tron Cells',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'creationDate': 1729267720,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612'},
 {'id': 5,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Corrupted Cyborg',
  'creator': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612',
  'creationDate': 1729267724,
  'owner': '0x43ADAc5516f8E2D3d2BD31276BeC343547ee6612'},
 {'id': 6,
  'collectionId': 1,
  'levelOfOwnership': 6,
  'name': 'Skynet',


In [33]:
nft_contract.functions.getCollectionNFTs(1).call()

[1, 2, 3, 4, 5, 6]